#ChatGPT on T5 base

In [1]:
!pip install transformers
!pip install sentencepiece

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [2]:

device = "cuda"

tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")
model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base").to(device)

def paraphrase(
    question,
    num_beams=5,
    num_beam_groups=5,
    num_return_sequences=5,
    repetition_penalty=10.0,
    diversity_penalty=3.0,
    no_repeat_ngram_size=2,
    temperature=0.7,
    max_length=250
):
    input_ids = tokenizer(
        f'paraphrase: {question}',
        return_tensors="pt", padding="longest",
        max_length=max_length,
        truncation=True,
    ).input_ids
    input_ids = input_ids.to(device)

    outputs = model.generate(
        input_ids, temperature=temperature, repetition_penalty=repetition_penalty,
        num_return_sequences=num_return_sequences, no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams, num_beam_groups=num_beam_groups,
        max_length=max_length, diversity_penalty=diversity_penalty
    )

    res = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return res


In [3]:
text = 'Curiosity can help you fuel your natural enthusiasm to explore and discover, stay open, and follow your passion. Staying curious is important for becoming and staying creative. Good news - it can be practiced!'

In [4]:
paraphrase(text)

["Being curious can help you tap into your natural talent of discovering and exploring, remain open, and pursue your passion. Staying curious is key to becoming and staying creative. Don't worry - there's hope!",
 "Being curious can help you tap into your natural talent of discovering and exploring, remain open, and pursue your passion. Staying curious is key to becoming and staying creative. Don't worry - there's hope! Thank you!",
 'To keep yourself open to discovering and exploring, cultivate curiosity. Being curious is crucial for developing and maintaining creativity; however, one must not be ignorant of the fact that it can be done!',
 'To keep yourself open to discovering and exploring, cultivate curiosity. Being curious is crucial for developing and maintaining creativity; however, one must not be ignorant of the fact that it can be done!!',
 "Keeping open to new things can encourage curiosity and exploring your natural instincts. It's essential to remain curious about the worl

#Pegasus

In [15]:
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

def get_response(input_text,num_return_sequences,num_beams):
  batch = tokenizer([input_text],truncation=False,padding='longest',max_length=200, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=200,num_beams=num_beams, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

In [16]:
num_beams = 10
num_return_sequences = 10
context = "Curiosity can help you fuel your natural enthusiasm to explore and discover, stay open, and follow your passion. Staying curious is important for becoming and staying creative. Good news - it can be practiced!"
# text = ''
get_response(context,num_return_sequences,num_beams)


['Staying curious can help fuel your enthusiasm to explore and discover.',
 'Staying curious can help you fuel your enthusiasm to explore and discover.',
 'Staying curious can help you discover, stay open, and follow your passion.',
 'Staying curious can fuel your enthusiasm to explore and discover.',
 'Staying curious can help fuel your enthusiasm to explore, stay open, and follow your passion.',
 'Staying curious can help you fuel your enthusiasm to explore and discover, stay open, and follow your passion.',
 "It's important to stay open and curious in order to become and stay creative.",
 'Staying curious can help you fuel your enthusiasm to explore and discover, stay open and follow your passion.',
 'Staying curious can help fuel your enthusiasm to explore, stay open and follow your passion.',
 "It's important to stay open and curious in order to stay creative."]

In [11]:
get_response(text, num_return_sequences, num_beams)


['Good news - it can be practiced!',
 'Good news - it can be practiced.',
 'Good news, it can be practiced.',
 'It can be practiced to stay curious.',
 'Good news, it can be practiced!',
 'Staying curious can be practiced.',
 'Good news - it is possible to practice.',
 "It's important to stay curious for becoming and staying creative.",
 'Good news - you can practice being curious.',
 'Good news - it can be done.']

#T5 for paraphrasing

#ChatGPT with a paraphrasing prompt


In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 6.2 MB/s eta 0:00:00


In [2]:
import openai
import torch

# INSERT API KEY HERE
# openai.api_key = ""

prompts = ["Provide 15 alternative sentences that are not too formal rewritings of the following paragraph: Curiosity can help you fuel your natural enthusiasm, openness to new domains and interest in pursuing them. Staying curious is important for creativity, and you can also practice it!",
           "Provide 15 alternative sentences that are not too formal rewritings of the following paragraph: I have several exercises for practicing curiosity for you. Select the one you would like to try.",
           "Provide 15 alternative sentences that are not too formal rewritings of the following paragraph: Congratulations, practicing curiosity will make you more inclined towards creativity!",
           "Provide 15 alternative sentences that are not too formal rewritings of the following paragraph: Would you like to try another exercise for practicing curiosity?",
           "Provide 15 alternative sentences that are not too formal rewritings of the following paragraph: Humor encourages a playful mindset, fosters unconventional thinking, and creates a relaxed atmosphere where innovative ideas can flourish. The following exercises are primarily based on the Incongruity Theory - we laugh when something non-serious violates our common belief, e.g., in punchline of a joke  (Aristotle, Kant, Schopenhauer).",
           "Provide 15 alternative sentences that are not too formal rewritings of the following paragraph: See below for several humour exercises. Choose one!",
           "Provide 15 alternative sentences that are not too formal rewritings of the following paragraph: Good job! Becoming more humorous will help you become creative!",
           "Provide 15 alternative sentences that are not too formal rewritings of the following paragraph: Would you like to try another exercise in the Humour category?"
]

count = 0
for prompt in prompts:
  count += 1

  # generate response using chat gpt
  response = openai.Completion.create(
            engine='text-davinci-003',
            prompt=prompt,
            temperature=0.7,
            max_tokens=1500,
            n=1,
            stop=None)

  # take the message
  message = response.choices[0].text.strip()
  print(message)

  # Opening a CSV file in write mode
  with open('output'+str(count) +'.txt','w') as file:
    # Using csv.writer to write the list to the CSV file
    writer = file.write(message)
    file.close()


1. Stirring up your curiosity can kindle your natural eagerness, willingness to explore different areas and fascination in exploring them.
2. Being inquisitive is essential for inventiveness, and you can also cultivate it!
3. Firing up your inquisitiveness can ignite your inherent enthusiasm, readiness to dive into various fields and fondness for investigating them.
4. Keeping nosy is significant for creativity, and you can also hone it!
5. Sparking your interest can arouse your innate fervor, openness to unfamiliar territories and zest for delving into them.
6. Being inquiring is critical for imagination, and you can also sharpen it!
7. Arousing your curiosity can kindle your native excitement, acceptance of various realms and relish for examining them.
8. Staying curious is indispensable for inventiveness, and you can also exercise it!
9. Reviving your inquisitiveness can rouse your inborn zeal, receptiveness to dissimilar domains and delight in studying them.
10. Being inquiring is 

In [24]:
print(type(message))

<class 'str'>
